# Run Model 1
Dit notebook kan gebruikt worden om BrfRFE_model1 te draaien. Dit model is bedoeld om te voorspellen of een kandidaat starter zal worden op basis van de data die verzamelt is voordat het eerste telefonisch contact is geweest. 

## Install packages

In [1]:
##

## Import Libraries

In [2]:
import pandas as pd
import numpy as np
import os
from helper import get_most_recent_file
from data_prep import run_data_prep


## Upload data
Je kan de data van de kandidaten waarover je wilt voorspellen of ze starter worden uploaden in de map "kandidaat_data". Dit dient in de vorm van een excel te zijn met de naam "kandidaten_[DATUM].xlsx". Op de plek van [DATUM] vul je de datum in van uploaden in het format yyyymmdd, zonder streepjes of iets er tussen. Bijvoorbeeld een data bestand geupload op 11 augustus 2024 heeft de naam "kandidaten_20240811.xlsx". 

De excel heeft een aantal vaste kolommen en kolomnamen. Een voorbeeld excel bevind zich in kandidaten_data/voorbeeld_kandidaten.xlsx. Let op dat wijzigingen in de namen van kolommen of antwoordcategorieen impact hebben op de bruikbaarheid van de data en de kwaliteit van het model. 

## Lees data in 

In [3]:
kandidaten_dir = r"kanidaat_data"
file_path = get_most_recent_file(kandidaten_dir)
df_kandidaten = pd.read_excel(file_path)

## Lees model in

In [6]:
import json
model_name = "model_1.pkl"
model_dir = "models"
model_path = os.path.join(model_dir, model_name)
meta_path = os.path.join(model_dir, model_name[:-4]+"_meta.json")
brf_rfe = pd.read_pickle(model_path)
with open(meta_path, 'r') as file:
    model_meta = json.load(file)

## Prepareer data voor forecasting

In [7]:
df_model_input = run_data_prep(df_kandidaten)

removed 0 candidates who did not have prioriteit 1
removed 0 candidates who did not have NULL for afwijsBasisGegevens


In [8]:
#select only model variables
from data_prep import prepare_input_dataframe
df_model_input = prepare_input_dataframe(df_model_input, model_meta['selected_features'])

In [9]:
df_model_input.fillna(999, inplace=True)

In [13]:
predictions = brf_rfe.predict(df_model_input)
probas = brf_rfe.predict_proba(df_model_input)

c:\Users\peter\anaconda3\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but BalancedRandomForestClassifier was fitted without feature names
  warnings.warn(
c:\Users\peter\anaconda3\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but BalancedRandomForestClassifier was fitted without feature names
  warnings.warn(


In [16]:
probas[:,1]

array([0.495, 0.495, 0.61 ])

In [21]:
from datetime import datetime
import re
model_predictions = pd.DataFrame([list(df_kandidaten["id"]),predictions,probas[:,1]],columns=["id","predictions","probabilities"])
datetimestring = re.sub(r'\D', '', str(datetime.today()))
output_path = f"results/model_predictions_{datetimestring}.xlsx"
model_predictions.to_excel(output_path)